# Lab 5: Naive Bayes and Random Forests
In this lab we'll learn to use [GridSearchCV] and take a closer look the [Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB) and [Random Forests](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) classifiers.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.cross_validation import train_test_split
%matplotlib inline

# Naive Bayes
Naive Bayes widely known for being effective for spam email classification. We'll try our hand at applying NB on the [Spam email dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) from the UCI data repository. Download the dataset and move it to your lab data directory.

In [4]:
fname = 'data/SMSSpamCollection'
df = pd.read_table(fname, names=['class', 'text'])
df.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Multinomial Naive Bayes
Given collection of emails $d^{(1)}, d^{(2)}, ..., d^{(N)}$ with corresponding labels $y^{(i)}$, we first need to extract features from each document. We'll do this by computing word counts of each word in each document. This is commonly referred to as a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) representation.

After doing some preprocessing of our data, we'll have: $\{(x^{(i)}, y^{(i)})\}_{i=1}^N$, where:
* $x^{(i)} \in R^d$ is a word count feature vector for document $i$, and $d$ is the size of the vocabulary. For the purposes of this lab, we define our vocabulary to be the set of words that appear in our dataset of documents.
$$ x^{(i)} = (\theta^{(i)}_1, \theta^{(i)}_2, ..., \theta^{(i)}_d)$$
$$\theta^{(i)}_k = \text{ the number of times word $k$ appears in document $i$}$$


* $y^{(i)} \in \{0, 1\}$ is its class, where $0$ denotes spam, and $1$ denotes real email.

First we need to compute word count feature vectors. We can do this manually like so:

In [5]:
import string

def get_all_words(docs):
    '''
    docs: pandas.Series of strings
    Returns: set of words(strings)
    '''
    # Compute all the words in the docs
    words = set()
    for d in docs:
        # this will strip punctuation
        # ref: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
        d = d.translate(None, string.punctuation)
        words = words.union(set(d.split()))
    return words

def make_word_cnt_feats(docs, vocabulary=None):
    '''
    docs: iterable(ie: list or pandas.Series) of strings
    vocabulary: set of words(strings) that appear in docs
    Returns: numpy matrix of the word count features
    '''
    if vocabulary is None:
        vocabulary = get_all_words(docs)
    words = sorted(vocabulary)
    word_dict = {w: idx for idx, w in enumerate(words)}
    
    # feature matrix will be of size n x (size of vocabulary + 1)
    # add 1 to account for words that dont appear in the vocabulary to avoid random issues
    vocab_size = len(words)
    word_cnt_features = np.zeros((len(docs), len(words) + 1))
    
    for idx in range(len(docs)):
        doc = docs[idx]
        doc = doc.translate(None, string.punctuation)
        words = doc.split()
        for w in words:
            word_idx = word_dict.get(w, vocab_size)
            word_cnt_features[idx, word_idx] += 1
            
    return word_cnt_features

In [6]:
vocabulary = get_all_words(df['text'])
feature_matrix = make_word_cnt_feats(df['text'], vocabulary)
print('Feature matrix shape: {}'.format(feature_matrix.shape))

Feature matrix shape: (5572, 11748)


It turns out that sklearn actually has a built-in method to do this kind of word count featurization! (But of course, it's good to get your hands dirty with manual featurization from time to time to understand all the ingredients of these algorithms). See sklearn's [sklearn.feature_extraction.text.CountVectorizer](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) function for more details.                                                                 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
word_cnt_feats = count_vectorizer.fit_transform(df['text'])
word_cnt_feats

<5572x8713 sparse matrix of type '<type 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) has a bunch of useful functions that you should checkout. The important ones that we'll use are:
* [fit](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit): this learns the vocabulary of the input collection of strings/"documents"
* [transform](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.transform): given a collection of documents, this produces the word count features for the documents
* fit_transform: calls fit, then transform

Note that the word count feature matrix(aka document-term matrix) is in sparse format. This is not an issue for the sklearn models. But if you want to expand them to non-sparse numpy arrays, use the toarray function. Note that this can be problematic for bigger datasets! We can somewhat get around this b/c this dataset is small enough that we can fully expand a 5k by 8k matrix(since it's sparse enough).

In [10]:
X = word_cnt_feats.toarray()
Y = (df['class'] == 'ham')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)

Now that we have the data in the document-term matrix format, we can use the Naive Bayes classifier!

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
nb = MultinomialNB(alpha=1)
nb.fit(X_train, y_train)
preds = nb.predict(X_test)

In [12]:
print('Accuracy: {:.3f}'.format(accuracy_score(preds, y_test)))
print('F1 Score: {:.3f}'.format(f1_score(preds, y_test)))
print('ROC AUC Score: {:.3f}'.format(roc_auc_score(preds, y_test)))

Accuracy: 0.983
F1 Score: 0.990
ROC AUC Score: 0.959


## Grid Search Cross Validation

Note that the code above fixed alpha to 1. But as we've learned in class and last week's lab, we should use cross validation for any kind of parameter search. Last week we wrote code to manually compute model performance across folds to better understand the full cross validation process. Now that we have that under our belt, going foward, we'll simply use sklearn's [GridSearchCV class](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), which will do most of this cumbersome work for us.

GridSearchCV requires:
* an estimator object(IE: LogisticRegression(), SVC(), KNeighborsClassifier(), etc)
* param_grid: a dictionary mapping parameter names to lists of values to search over. We'll see an example below
* scoring: a string. See the valid inputs here: http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
* n_jobs: number of jobs to run in parallel to speed up the computation. GridSearchCV can take a long time if you have a lot of different parameter combinations to try out. So it's important to set this parameter if your laptop/desktop has a lot of cores that you want to take advantage of.
* cv: int, number of cross validation folds to use
* refit: boolean, if true, at the end of the Grid Search, GridSearchCV will refit the entire data with a model using the optimal parameters(you can then access this with the GridSearchCV.best\_parameters_ attribute).

Here's an example of how we'd use it for some random data:

In [31]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

def sample_cross_val_param_search():
    # parameters to search over
    penalties = ['l1', 'l2']
    c_vals = [10**i for i in range(-2, 3)]
    param_grid = {'penalty': penalties, 'C': c_vals}
    # make some fake data
    n_samples = 1000
    feats = 10
    x_train = np.random.random((n_samples, feats))
    y_train = (np.random.random(n_samples) > 0.5) # randomly generate True/False labels
    
    grid = GridSearchCV(LogisticRegression(), param_grid, 'f1', cv=5)
    grid.fit(x_train, y_train)
    return grid

In [42]:
grid = sample_cross_val_param_search()
results = pd.DataFrame(grid.cv_results_)
results.head(3)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_penalty,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.001166,0.001173,0.000000,0.000000,0.01,l1,"{u'penalty': u'l1', u'C': 0.01}",9,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000121,0.000094,0.000000,0.000000
1,0.001685,0.001012,0.510124,0.536665,0.01,l2,"{u'penalty': u'l2', u'C': 0.01}",8,0.505495,0.522546,...,0.431818,0.546433,0.587678,0.567442,0.547368,0.529801,0.000080,0.000065,0.053982,0.018286
2,0.001067,0.000889,0.000000,0.000000,0.1,l1,"{u'penalty': u'l1', u'C': 0.1}",9,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000437,0.000030,0.000000,0.000000


In [34]:
print('Best estimator: {}'.format(grid.best_estimator_))
print('Best Parameters: {}'.format(grid.best_params_))

Best estimator: LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Best Parameters: {'penalty': 'l2', 'C': 0.01}


## Naive Bayes Tasks
Now that we've seen how to use GridSearchCV, it's your turn to use it on the Naive Bayes model. Pick an evaluation metric to use for cross validation like F1 score, ROC AUC score, etc. In the spam classification setting, do you think it makes sense to optimize for higher precision? recall? f1 score?

0) Pick an evaluation metric.

1) Write some code to use GridSearchCV to learn the best alpha parameter for your Multinomial Naive Bayes model.

In [40]:
# CODE

2) Now that you've found the optimal alpha using cross validation over the training set, evaluate the model on the test set.

In [41]:
# CODE

Further Questions:
* Naive Bayes has a seemingly stringent independence assumption. Do you think the variables in this email classification problem(word count vectors) are really independent? Does this matter? Why/why not?

## Random Forests
Two weeks ago, we played around with decision trees. In general, practictioners almost never use decision trees over Random Forests. Recall from lecture, that Random Forests takes some number of subsamples of the data:
$S_1, S_2, ..., S_k \subseteq \{(x^{(i)}, y^{(i)}\}_{i=0}^N$. On each subsample of the data, $S_i$, the Random Forests algorithm trains a decision tree on $S_i$.

At classification time, when we get a datapoint $x$, the Random Forests classifier runs $x$ through decision trees $D_1, D_2, ..., D_k$, which produces a "probability" estimate of each class:

$$P(y = 1 | x) = \frac{\text{number of decision trees that classified $x$ as 1}}{k}$$

$$P(y = 0 | x) = \frac{\text{number of decision trees that classified $x$ as 1}}{k}$$

Then from there, we get a class prediction by picking $y = 1$ if $P(y = 1 | x) > T$, for some threshold probability $T$.


Instead of using the spam dataset to try out Random Forests, we'll re-use the credit loan data from Assignment 2.

In [43]:
# load data

Do whatever data cleaning, data imputing, etc you need to do.

In [44]:
# clean data

Split the data into train/test sets.

In [46]:
# split data

## Random Forest Parameters

The Random Forest parameters are more or less the same as those for Decision Trees. The only additional parameter is n_estimators, which is the number of decision trees to grow for the algorithm.
* n_estimators : integer, the number of trees in the forest.
* criterion : string,  “gini” or “entropy” 
* max_features : int, float, string or None, the number of features to consider when looking for the best split
* max_depth : integer, the maximum depth of the tree
* min_samples_split : int, float, the minimum number of samples required to split an internal node:
* min_samples_leaf : int, float, optional (default=1), the minimum number of samples required to be at a leaf node:
* min_weight_fraction_leaf : float, optional 
* max_leaf_nodes : int or None, optional (default=None)

0) What do you think will be the effect of increasing the n\_estimators parameter? If n_estimators = 1, is this any different to using a decision tree classifier(assuming you set all the other parameters like min_sample_split, max_depth, etc the same way in both cases)?

There are a lot of parameters here. Imagine writing a 8x nested for-loop to find all the combinations of parameters for a RandomForests model. That doesn't sound too fun right? Good thing we have GridSearchCV.

0) Pick an evaluation metric.

1) Pick a few values for each of the random forest parameters for the param\_grid. Note that this Grid Search can be a lot more time consuming than the one you did for Naive Bayes because each Random Forest has to train multiple decision trees for each combination of parameters in your param_grid.

In [52]:
# param settings to try out

2) Write some code to do the grid search for Random Forests

In [49]:
# do the grid search

3) Random Forests has a [feature importances](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_) attribute. What features are the most important?

In [53]:
# code

4) Rerun the classifier with a different scoring metric. Do the feature importances change? Which features are most important now?

In [55]:
# code

## References
* https://etav.github.io/projects/spam_message_classifier_naive_bayes.html
* http://scikit-learn.org/stable/modules/naive_bayes.html
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html